<h2
    style="font-size:20px; font-family:'Helvetica Neue'" align="right"> Viktor Trokhymenko 
</h2>
<h3 
    style="font-size:16px; font-family:'Helvetica Neue'" align="right"> @vtrokhymenko
</h3>

In [44]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
np.random.seed(42)
import re
from pprint import pprint

import nltk
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

#%load_ext autoreload
#%autoreload 2

#### пока для примера беру лемматиатор от `pymorphy2` & `pymystem3`

* pymorphy2

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
#затестим лемматизатор
morph.parse('приехали')[0].normal_form

* pymystem3 (возьмем за основу)

In [2]:
from pymystem3 import Mystem

In [3]:
m=Mystem()

In [4]:
m.lemmatize('машиной')[0]

'машина'

#### добавим дополнительное(ые) стоп-слово(а), которое(ые) удалось заметить, но эти стоп-слова нужно по мере необходимости дополнять

In [9]:
stopword_ru=stopwords.words('russian')
len(stopword_ru)

151

In [10]:
stopword_ru.append('м')
stopword_ru.append('это') #примеры в этом ноутбуке пока с этим токеном6 увидил это слово после визуализации
print(len(stopword_ru))
print(stopword_ru)

152
['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 

#### читаем данные

In [11]:
#https://github.com/buriy/russian-nlp-datasets/releases
df = pd.read_csv('data/novayagazeta.ru.txt', sep='\t', encoding='utf-8', header=None)
df.columns = ['id', 'url', 'title', 'text']

print('before filter duplicates: ',df.shape)
df = df.drop_duplicates('url')
print('after filter duplicates: ',df.shape)
df.head()

before filter duplicates:  (14491, 4)
after filter duplicates:  (14485, 4)


,id,url,title,text
0,1408695844,http://www.novayagazeta.ru/columns/64935.html,Правительство аккуратно редактирует антисанкци...,"Как и предсказывал на страницах, а также на са..."
1,1408695844,http://www.novayagazeta.ru/politics/64938.html,"70 лет назад, в августе 44-го, Париж стал своб...",Август 1944. Освобожденные заключенные на площ...
2,1408695844,http://www.novayagazeta.ru/inquests/64932.html,«Меня назначили ответственным за теракт»,Август в России — время трагических годовщин. ...
3,1408695844,http://www.novayagazeta.ru/politics/64933.html,Зотов VS Клычков,В 21-м избирательном округе (районы Выхино-Жул...
4,1408695844,http://www.novayagazeta.ru/columns/64934.html,Экономику пропагандой не обманешь,"1. Белоруссия и Казахстан, с которыми у России..."


In [14]:
df.loc[4].text

'1. Белоруссия и Казахстан, с которыми у России таможенный союз (что означает отсутствие таможенного контроля на границе), не поддержали это решение. Следовательно, те же товары, но гораздо дороже, будут приходить в Россию как контрабанда. Думаю, никто в этом и не сомневается. По официальным опубликованным данным за пять месяцев 2014 года, белорусских лимонов в Россию поставлено на $1,7 млн, белорусских бананов — на $183 000, белорусских ананасов — на $37 000, белорусских кокосовых и миндальных орехов — почти на $1,8 млн, белорусских мидий — на $792 000, белорусских осьминогов — на $449 000.\\n2. Все сделанные в последнее время и уже опубликованные расчеты показывают, что в результате этих «ответных санкций» вырастут цены на продовольствие в наших магазинах. Известно, что на поставки из стран, в отношении которых наложен запрет, приходится порядка 20% от общего объема потребления. Одной из главных причин роста цен станет сокращение поставок мяса, тем более что цены на мясо на мировом р

#### ф-я для чистки текста: 
* от мусора и 
* удаляем стоп-слова

In [12]:
def clean_text(text): 
    
    #убираем мусор
    text = nltk.re.sub('-\s\r\n\s+|-\s\r\n|\r\n|[«»]|[><]', ' ', text.lower())    
    text = nltk.re.sub('[0-9]|[.,:;_%©?*!@#№$^•·&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-|–|—|', ' ', text)
    text = re.sub(r"\d+", ' ', text, flags=re.UNICODE)
    text = re.sub('\’|\”|‘', ' ', text)
    text = re.sub(r'\\n', '', text)
    text = re.sub('ё', 'е', text)
    
    #убираем стоп-слова
    words = text.split()
    words = [w for w in words if not w in stopword_ru]
    
    #джойним слова
    return ' '.join(words)

clean_text(df.loc[4].text)

'белоруссия казахстан которыми россии таможенный союз означает отсутствие таможенного контроля границе поддержали это решение следовательно те товары гораздо дороже будут приходить россию контрабанда думаю никто сомневается официальным опубликованным данным пять месяцев года белорусских лимонов россию поставлено млн белорусских бананов белорусских ананасов белорусских кокосовых миндальных орехов млн белорусских мидий белорусских осьминогов сделанные последнее время опубликованные расчеты показывают результате этих ответных санкций вырастут цены продовольствие наших магазинах известно поставки стран отношении которых наложен запрет приходится порядка общего объема потребления одной главных причин роста цен станет сокращение поставок мяса цены мясо мировом рынке растут результате рост цен ближайшие полгода составит решение запрете ввоз продуктов принято условиях российская экономика осени минувшего года резко тормозит итогам первой половины года вошла рецессию т е начался экономический с

In [13]:
%%time

df['text_clear']=df.text.apply(clean_text)

CPU times: user 13.9 s, sys: 98 ms, total: 14 s
Wall time: 14 s


In [15]:
df.head()

,id,url,title,text,text_clear
0,1408695844,http://www.novayagazeta.ru/columns/64935.html,Правительство аккуратно редактирует антисанкци...,"Как и предсказывал на страницах, а также на са...",предсказывал страницах также сайте новой иван ...
1,1408695844,http://www.novayagazeta.ru/politics/64938.html,"70 лет назад, в августе 44-го, Париж стал своб...",Август 1944. Освобожденные заключенные на площ...,август освобожденные заключенные площади оперы...
2,1408695844,http://www.novayagazeta.ru/inquests/64932.html,«Меня назначили ответственным за теракт»,Август в России — время трагических годовщин. ...,август россии время трагических годовщин десят...
3,1408695844,http://www.novayagazeta.ru/politics/64933.html,Зотов VS Клычков,В 21-м избирательном округе (районы Выхино-Жул...,избирательном округе районы выхино жулебино ря...
4,1408695844,http://www.novayagazeta.ru/columns/64934.html,Экономику пропагандой не обманешь,"1. Белоруссия и Казахстан, с которыми у России...",белоруссия казахстан которыми россии таможенны...


In [21]:
df.loc[np.random.randint(low=0, high=df.shape[0])].text_clear

'австрия германия согласились принять своих территориях беженцев ближнего востока которые находятся венгрии сообщил своем facebook канцлер австрии вернер файман сказано сообщении решение принято встречи венгерским премьер министром виктором орбаном координации немецким канцлером ангелой меркель кроме файн выразил надежду венгрия выполнит свои обязательства числе дублинской конвенции согласно которой беженцев отвечает та страна которой оказались первую очередь читайте также германия официально признала проблему беженцев приток которых страну год вырос четыре разаранее венгерские власти обещали выделить мигрантам которые пешком собирались идти будапешта австрийской границы около ста автобусов данным источников границе австрии направляются около мигрантов накануне премьер министр дэвид кэмерон заявил великобритания готова принять тысячи мигрантов сирии которых распределят специальные лагеря беженцев конце августа венгрия установила границе сербией колючую проволоку надеясь подобные меры п

#### применяем лемматизатор

In [14]:
%%time

#df['text_clear_lemma'] = df['text_clear'].astype(str).apply(lambda x: [morph.parse(i)[0].normal_form for i in x.split()])
df['text_clear_lemma'] = df['text_clear'].astype(str).apply(lambda x: [m.lemmatize(i)[0] for i in x.split()])

CPU times: user 1min 44s, sys: 38.5 s, total: 2min 22s
Wall time: 11min 5s


In [28]:
df.head()

,id,url,title,text,text_clear,text_clear_lemma
0,1408695844,http://www.novayagazeta.ru/columns/64935.html,Правительство аккуратно редактирует антисанкци...,"Как и предсказывал на страницах, а также на са...",предсказывал страницах также сайте новой иван ...,"[предсказывать, страница, также, сайт, новый, ..."
1,1408695844,http://www.novayagazeta.ru/politics/64938.html,"70 лет назад, в августе 44-го, Париж стал своб...",Август 1944. Освобожденные заключенные на площ...,август освобожденные заключенные площади оперы...,"[август, освобождать, заключенная, площадь, оп..."
2,1408695844,http://www.novayagazeta.ru/inquests/64932.html,«Меня назначили ответственным за теракт»,Август в России — время трагических годовщин. ...,август россии время трагических годовщин десят...,"[август, россия, время, трагический, годовщина..."
3,1408695844,http://www.novayagazeta.ru/politics/64933.html,Зотов VS Клычков,В 21-м избирательном округе (районы Выхино-Жул...,избирательном округе районы выхино жулебино ря...,"[избирательный, округ, район, выхино, жулебино..."
4,1408695844,http://www.novayagazeta.ru/columns/64934.html,Экономику пропагандой не обманешь,"1. Белоруссия и Казахстан, с которыми у России...",белоруссия казахстан которыми россии таможенны...,"[белоруссия, казахстан, который, россия, тамож..."


In [15]:
df.to_csv('data/novayagazeta_clear_pymystem3.csv',index=False)

In [23]:
print(df.loc[np.random.randint(low=0, high=df.shape[0])].text_clear_lemma)

['мосгорсуд', 'прокомментировать', 'просьба', 'навальный', 'отпускать', 'похороны', 'немцовавый', 'московский', 'городской', 'суд', 'прокомментировать', 'просьба', 'оппозиционный', 'политика', 'алексей', 'навальный', 'отпускать', 'спецприемник', 'похороны', 'убивать', 'политика', 'борис', 'немцов', 'передавать', 'интерфакс', 'руководитель', 'пресс', 'служба', 'судно', 'ульяна', 'солопов', 'сообщать', 'агентство', 'решение', 'допуск', 'арестованный', 'политика', 'похороны', 'немцов', 'принимать', 'суд', 'назначать', 'навальный', 'сутки', 'арест', 'пока', 'слово', 'солоповый', 'заявление', 'навальный', 'судно', 'глава', 'пресс', 'служба', 'мосгорсуд', 'рассказывать', 'экстренный', 'случай', 'административный', 'арест', 'приостанавливать', 'ранее', 'навальный', 'свой', 'страница', 'facebook', 'заявлять', 'надеяться', 'прощаться', 'борис', 'немцов', 'несмотря', 'административный', 'арест', 'срок', 'заключение', 'агитационный', 'акция', 'метро', 'навальный', 'заканчиваться', 'март', 'читать

## LDA

In [24]:
from gensim import corpora, models

* присваиваем каждому токену уникальный идентификатор, то есть `id`

In [25]:
%%time

dictionary = corpora.Dictionary(df['text_clear_lemma'].values)

CPU times: user 4.09 s, sys: 37.1 ms, total: 4.13 s
Wall time: 4.14 s


In [26]:
print(len(dictionary.token2id))
dictionary.token2id

#не особо понимаю, почему "ё" есть в словах, если я делал в очистке ф-и такое `re.sub('ё', 'е', text)` и отдельно работает

106171


{'автопром': 0,
 'аккуратно': 1,
 'аккуратный': 2,
 'акселерат': 3,
 'активный': 4,
 'амнистировать': 5,
 'аналог': 6,
 'аналогия': 7,
 'анекдот': 8,
 'антисанкция': 9,
 'бада': 10,
 'бедно': 11,
 'безлактозный': 12,
 'безусловно': 13,
 'белорусский': 14,
 'белье': 15,
 'бесчеловечный': 16,
 'биологически': 17,
 'болид': 18,
 'большой': 19,
 'бородатый': 20,
 'буш': 21,
 'быстрый': 22,
 'быть': 23,
 'бюджет': 24,
 'вводить': 25,
 'ведомство': 26,
 'вещь': 27,
 'виагра': 28,
 'видимо': 29,
 'возвращение': 30,
 'вредный': 31,
 'вспомнить': 32,
 'вто': 33,
 'второй': 34,
 'вывешивать': 35,
 'выводить': 36,
 'выкапывать': 37,
 'выпадение': 38,
 'выясняться': 39,
 'геннадий': 40,
 'гибридный': 41,
 'гмо': 42,
 'год': 43,
 'голова': 44,
 'голос': 45,
 'горох': 46,
 'госаппарат': 47,
 'гражданин': 48,
 'давать': 49,
 'два': 50,
 'действие': 51,
 'действительность': 52,
 'дело': 53,
 'диабет': 54,
 'добавка': 55,
 'должно': 56,
 'должный': 57,
 'доллар': 58,
 'другой': 59,
 'еда': 60,
 'женски

* делаем список преобразованных текстов или же "To actually convert tokenized documents to vectors"

In [27]:
%%time

doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['text_clear_lemma'].values]

CPU times: user 3.15 s, sys: 134 ms, total: 3.28 s
Wall time: 3.29 s


In [28]:
print(doc_term_matrix[123])

[(43, 5), (50, 3), (53, 1), (64, 2), (68, 1), (83, 1), (174, 1), (183, 1), (192, 1), (223, 1), (313, 1), (388, 1), (516, 1), (581, 1), (601, 1), (614, 1), (659, 1), (695, 1), (807, 1), (903, 1), (927, 1), (945, 1), (1129, 1), (1133, 1), (1161, 1), (1163, 2), (1212, 3), (1228, 1), (1496, 1), (1514, 1), (1575, 2), (1643, 2), (1644, 1), (1750, 1), (1786, 1), (1789, 1), (1799, 1), (1916, 1), (1941, 1), (1969, 1), (2075, 1), (2128, 1), (2335, 1), (2613, 1), (2615, 1), (2643, 1), (2717, 1), (2727, 1), (2744, 1), (2804, 1), (2809, 1), (2854, 2), (2949, 3), (3002, 1), (3024, 1), (3180, 1), (3222, 1), (3464, 1), (3805, 1), (3907, 1), (3909, 1), (3922, 1), (3942, 1), (3948, 1), (4162, 1), (4629, 2), (4637, 1), (4668, 1), (4670, 1), (4732, 3), (4758, 1), (4804, 1), (4806, 2), (5018, 3), (5696, 1), (6622, 2), (6792, 1), (8120, 5), (8121, 1), (8122, 2), (8123, 3), (8124, 4), (8125, 1), (8126, 1), (8127, 1), (8128, 1)]


In [29]:
print(df.text_clear_lemma[123])
print(doc_term_matrix[123])

['суд', 'приговаривать', 'захватчик', 'белгородский', 'банка', 'два', 'год', 'колония', 'поселение', 'октябрьский', 'суд', 'белгород', 'приговаривать', 'александра', 'вдовина', 'захватить', 'офис', 'банка', 'западный', 'два', 'год', 'колония', 'поселение', 'передавать', 'итар', 'тасс', 'судья', 'признавать', 'вдовина', 'виновный', 'самоуправство', 'незаконный', 'лишение', 'свобода', 'два', 'лицо', 'применение', 'оружие', 'срок', 'отбывание', 'наказание', 'случай', 'вдовина', 'отсчитываться', 'апрель', 'год', 'мужчина', 'арестовывать', 'обвинитель', 'запрашивать', 'судно', 'вдовина', 'год', 'месяц', 'колония', 'поселение', 'сторона', 'защита', 'настаивать', 'наказание', 'связанный', 'лишение', 'свобода', 'удовлетворять', 'вынести', 'приговор', 'процесс', 'дело', 'захват', 'офис', 'банка', 'проходить', 'особый', 'порядок', 'исследование', 'доказательство', 'ходатайство', 'обвиняемый', 'такой', 'образ', 'вдовин', 'полностью', 'признавать', 'вина', 'напоминать', 'апрель', 'год', 'отзыв', '

* tf-idf

In [30]:
%%time 

tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]

CPU times: user 795 ms, sys: 7.62 ms, total: 803 ms
Wall time: 802 ms


In [31]:
print(corpus_tfidf[123])

[(43, 0.04337238700035509), (50, 0.07674836319406926), (53, 0.015403375226679566), (64, 0.08984819034754493), (68, 0.03781021746018334), (83, 0.006760829768534811), (174, 0.024500859341925176), (183, 0.022373457326028807), (192, 0.027561586705752116), (223, 0.01869848309160451), (313, 0.044075776604810935), (388, 0.01474066164796024), (516, 0.06072594376064386), (581, 0.029594665941597296), (601, 0.031167763205994917), (614, 0.04499842823068974), (659, 0.0349153467313694), (695, 0.07272117596755573), (807, 0.03149878896954517), (903, 0.026491927000761136), (927, 0.027958115783119148), (945, 0.02501142421190265), (1129, 0.037366240666900946), (1133, 0.06645760492857024), (1161, 0.06534702814027844), (1163, 0.10230920525889563), (1212, 0.12833096440927402), (1228, 0.02612427706051128), (1496, 0.03188574658950131), (1514, 0.03215735426213943), (1575, 0.06309329284239575), (1643, 0.05106693909627816), (1644, 0.036492031736103055), (1750, 0.04145670157776293), (1786, 0.03878753775201464), (

### LDA

* ### gensim

In [47]:
import pyLDAvis.gensim
from gensim.models import CoherenceModel, LdaMulticore

In [48]:
%%time

lda_model = LdaMulticore(doc_term_matrix,
                         num_topics=30,
                         id2word=dictionary,
                         workers=8)

Process ForkPoolWorker-30:
Process ForkPoolWorker-26:
Process ForkPoolWorker-25:
Process ForkPoolWorker-28:
Process ForkPoolWorker-29:
Process ForkPoolWorker-27:
Process ForkPoolWorker-20:
Process ForkPoolWorker-22:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/trokhymenko/anaconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/trokhymenko/anaconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/trokhymenko/anaconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/trokhymenko/anaconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
   

KeyboardInterrupt: 

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
%%time
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(ldamodel_fast,
                                doc_term_matrix,
                                dictionary,
                                mds='tsne')

* tf_idf

In [58]:
lda_model_tfidf = models.LdaMulticore(corpus_tfidf,
                                      num_topics=30,
                                      id2word=dictionary,
                                      passes=1,
                                      workers=6)

Process ForkPoolWorker-72:
Process ForkPoolWorker-66:
Process ForkPoolWorker-71:
Process ForkPoolWorker-69:
Process ForkPoolWorker-63:
Process ForkPoolWorker-70:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", li

KeyboardInterrupt: 

Process ForkPoolWorker-87:
Process ForkPoolWorker-77:
Process ForkPoolWorker-88:
Process ForkPoolWorker-78:
Process ForkPoolWorker-85:
Process ForkPoolWorker-76:
Process ForkPoolWorker-86:
Process ForkPoolWorker-89:
Process ForkPoolWorker-79:
Process ForkPoolWorker-74:
Process ForkPoolWorker-73:
Process ForkPoolWorker-84:
Process ForkPoolWorker-82:
Process ForkPoolWorker-83:
Process ForkPoolWorker-80:
Process ForkPoolWorker-75:
Process ForkPoolWorker-81:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-90:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self

  File "/Users/vtrokhymenko/env/py36/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 330, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
KeyboardInterrupt
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, 

  File "/Users/vtrokhymenko/env/py36/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 330, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Users/vtrokhymenko/env/py36/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 330, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
KeyboardInterrupt
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:


* ### sklearn

https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/

In [32]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

[Non-Negative Matrix Factorization](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html)

[Latent Dirichlet Allocation with online variational Bayes algorithm](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#sklearn.decomposition.LatentDirichletAllocation)



In [47]:
df.head()

,id,url,title,text,text_clear,text_clear_lemma
0,1408695844,http://www.novayagazeta.ru/columns/64935.html,Правительство аккуратно редактирует антисанкци...,"Как и предсказывал на страницах, а также на са...",предсказывал страницах также сайте новой иван ...,"[предсказывать, страница, также, сайт, новый, ..."
1,1408695844,http://www.novayagazeta.ru/politics/64938.html,"70 лет назад, в августе 44-го, Париж стал своб...",Август 1944. Освобожденные заключенные на площ...,август освобожденные заключенные площади оперы...,"[август, освобождать, заключенная, площадь, оп..."
2,1408695844,http://www.novayagazeta.ru/inquests/64932.html,«Меня назначили ответственным за теракт»,Август в России — время трагических годовщин. ...,август россии время трагических годовщин десят...,"[август, россия, время, трагический, годовщина..."
3,1408695844,http://www.novayagazeta.ru/politics/64933.html,Зотов VS Клычков,В 21-м избирательном округе (районы Выхино-Жул...,избирательном округе районы выхино жулебино ря...,"[избирательный, округ, район, выхино, жулебино..."
4,1408695844,http://www.novayagazeta.ru/columns/64934.html,Экономику пропагандой не обманешь,"1. Белоруссия и Казахстан, с которыми у России...",белоруссия казахстан которыми россии таможенны...,"[белоруссия, казахстан, который, россия, тамож..."


In [33]:
df['text_clear_lemma_1sentence']=df.text_clear_lemma.apply(lambda x: ' '.join(x))
df.head(1)

,id,url,title,text,text_clear,text_clear_lemma,text_clear_lemma_1sentence
0,1408695844,http://www.novayagazeta.ru/columns/64935.html,Правительство аккуратно редактирует антисанкци...,"Как и предсказывал на страницах, а также на са...",предсказывал страницах также сайте новой иван ...,"[предсказывать, страница, также, сайт, новый, ...",предсказывать страница также сайт новый иван с...


* #### TF DTM

In [ ]:
tf_vectorizer = CountVectorizer()

dtm_tf = tf_vectorizer.fit_transform(df.text_clear_lemma_1sentence)
print(dtm_tf.shape)

In [41]:
%%time

lda_tf = LatentDirichletAllocation(n_topics=30,               
                                   max_iter=2,                
                                   learning_method='online',
                                   random_state=42
                                  )

lda_tf.fit(dtm_tf)

/Users/trokhymenko/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 52 s, sys: 4.59 s, total: 56.6 s
Wall time: 56.4 s


In [45]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
21     0.286525 -0.116987       1        1  28.426496
15     0.353172 -0.064098       2        1  19.943490
4      0.346154  0.042431       3        1  17.427812
10     0.348636 -0.031857       4        1  10.291237
0      0.258679 -0.139265       5        1   6.181283
26     0.264848 -0.088106       6        1   3.849119
24     0.156977  0.274323       7        1   2.046981
8      0.109212  0.045676       8        1   1.725251
16     0.176277 -0.028307       9        1   1.521126
25     0.072423  0.336368      10        1   1.398815
22     0.109145 -0.025737      11        1   1.240625
14     0.054818 -0.021895      12        1   1.044404
9     -0.038037  0.072094      13        1   0.975467
11    -0.011203  0.062560      14        1   0.643045
18    -0.108514  0.017769      15        1   0.383253
28    -0.102563 -0.040785      16        1   0.315537
29    -0.137771 -0.007027      17        1   0.287749
7     -0.138201 -0.015763      18        1   0.232784
1     -0.137693 -0.006411      19        1   0.231704
6     -0.155047 -0.015423      20        1   0.216775
27    -0.159736 -0.018586      21        1   0.213778
3     -0.153782 -0.016637      22        1   0.212876
12    -0.155961 -0.019232      23        1   0.188456
20    -0.166136 -0.022309      24        1   0.172691
23    -0.169310 -0.019549      25        1   0.163539
13    -0.170251 -0.027007      26        1   0.153767
5     -0.178361 -0.028833      27        1   0.140037
19    -0.183707 -0.032104      28        1   0.134854
2     -0.182748 -0.030869      29        1   0.131432
17    -0.187848 -0.034434      30        1   0.105614, topic_info=       Category          Freq         Term         Total  loglift  logprob
term                                                                      
59413   Default  11340.000000        новый  11340.000000  30.0000  30.0000
80499   Default   7649.000000        рубль   7649.000000  29.0000  29.0000
24184   Default  33568.000000          год  33568.000000  28.0000  28.0000
80194   Default  22122.000000       россия  22122.000000  27.0000  27.0000
22437   Default   4642.000000       газета   4642.000000  26.0000  26.0000
100970  Default  19865.000000      человек  19865.000000  25.0000  25.0000
23820   Default   6731.000000        глава   6731.000000  24.0000  24.0000
42111   Default   4859.000000     компания   4859.000000  23.0000  23.0000
105158  Default  33088.000000          это  33088.000000  22.0000  22.0000
25195   Default   5269.000000  государство   5269.000000  21.0000  21.0000
80161   Default  12922.000000   российский  12922.000000  20.0000  20.0000
94832   Default   8350.000000      украина   8350.000000  19.0000  19.0000
78370   Default   5541.000000      ребенок   5541.000000  18.0000  18.0000
88699   Default  10631.000000       страна  10631.000000  17.0000  17.0000
81358   Default   3067.000000         сайт   3067.000000  16.0000  16.0000
88192   Default   3589.000000       статья   3589.000000  15.0000  15.0000
103048  Default   2385.000000        школа   2385.000000  14.0000  14.0000
81669   Default   2883.000000      самолет   2883.000000  13.0000  13.0000
19762   Default   5523.000000      военный   5523.000000  12.0000  12.0000
72602   Default   8155.000000    президент   8155.000000  11.0000  11.0000
62566   Default   5593.000000  организация   5593.000000  10.0000  10.0000
26650   Default   4887.000000       данный   4887.000000   9.0000   9.0000
67440   Default   2309.000000       письмо   2309.000000   8.0000   8.0000
27386   Default  14505.000000         дело  14505.000000   7.0000   7.0000
89198   Default   7664.000000          суд   7664.000000   6.0000   6.0000
84417   Default   2493.000000        сирия   2493.000000   5.0000   5.0000
86503   Default   8924.000000     сообщать   8924.000000   4.0000   4.0000
51874   Default   2495.000000          млн   2495.0000

In [52]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer,mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
21    -28.776649  16.459492       1        1  28.426496
15    -47.787971 -12.355244       2        1  19.943490
4     -10.173838 -12.195324       3        1  17.427812
10     38.274124  24.293341       4        1  10.291237
0      59.269260  14.073561       5        1   6.181283
26      5.234097  42.165878       6        1   3.849119
24    -11.182600  28.058750       7        1   2.046981
8       6.068460   0.994591       8        1   1.725251
16    -49.284023  10.814016       9        1   1.521126
25    -12.914138  48.603123      10        1   1.398815
22     51.839260 -14.129308      11        1   1.240625
14    -51.003124  34.130165      12        1   1.044404
9     -28.892847 -26.229546      13        1   0.975467
11     -9.264781  69.730141      14        1   0.643045
18    -29.653292  36.054062      15        1   0.383253
28     22.472239  35.173279      16        1   0.315537
29     39.420940   3.265174      17        1   0.287749
7       7.782950 -19.957253      18        1   0.232784
1      51.711498  40.873043      19        1   0.231704
6      32.602322 -27.453049      20        1   0.216775
27      5.869005  21.767908      21        1   0.213778
3     -27.768061  -3.201772      22        1   0.212876
12     21.555447  12.924331      23        1   0.188456
20    -34.512012  57.725498      24        1   0.172691
23    -10.455838   8.601067      25        1   0.163539
13     -9.294856 -35.879360      26        1   0.153767
5      32.912212  54.452065      27        1   0.140037
19     12.642324  62.379425      28        1   0.134854
2      23.584761  -8.147700      29        1   0.131432
17     13.707137 -41.055607      30        1   0.105614, topic_info=       Category          Freq         Term         Total  loglift  logprob
term                                                                      
59413   Default  11340.000000        новый  11340.000000  30.0000  30.0000
80499   Default   7649.000000        рубль   7649.000000  29.0000  29.0000
24184   Default  33568.000000          год  33568.000000  28.0000  28.0000
80194   Default  22122.000000       россия  22122.000000  27.0000  27.0000
22437   Default   4642.000000       газета   4642.000000  26.0000  26.0000
100970  Default  19865.000000      человек  19865.000000  25.0000  25.0000
23820   Default   6731.000000        глава   6731.000000  24.0000  24.0000
42111   Default   4859.000000     компания   4859.000000  23.0000  23.0000
105158  Default  33088.000000          это  33088.000000  22.0000  22.0000
25195   Default   5269.000000  государство   5269.000000  21.0000  21.0000
80161   Default  12922.000000   российский  12922.000000  20.0000  20.0000
94832   Default   8350.000000      украина   8350.000000  19.0000  19.0000
78370   Default   5541.000000      ребенок   5541.000000  18.0000  18.0000
88699   Default  10631.000000       страна  10631.000000  17.0000  17.0000
81358   Default   3067.000000         сайт   3067.000000  16.0000  16.0000
88192   Default   3589.000000       статья   3589.000000  15.0000  15.0000
103048  Default   2385.000000        школа   2385.000000  14.0000  14.0000
81669   Default   2883.000000      самолет   2883.000000  13.0000  13.0000
19762   Default   5523.000000      военный   5523.000000  12.0000  12.0000
72602   Default   8155.000000    президент   8155.000000  11.0000  11.0000
62566   Default   5593.000000  организация   5593.000000  10.0000  10.0000
26650   Default   4887.000000       данный   4887.000000   9.0000   9.0000
67440   Default   2309.000000       письмо   2309.000000   8.0000   8.0000
27386   Default  14505.000000         дело  14505.000000   7.0000   7.0000
89198   Default   7664.000000          суд   7664.000000   6.0000   6.0000
84417   Default   2493.000000        сирия   2493.000000   5.0000   5.0000
86503   Default   8924.000000     сообщать   8924.000000   4.0000

In [53]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer,mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
21     0.121538 -0.370192       1        1  28.426496
15     0.015606 -0.441609       2        1  19.943490
4     -0.056739 -0.436015       3        1  17.427812
10    -0.159886 -0.432425       4        1  10.291237
0      0.214576 -0.408174       5        1   6.181283
26    -0.284515 -0.347361       6        1   3.849119
24    -0.056087 -0.286313       7        1   2.046981
8      0.396078 -0.188704       8        1   1.725251
16    -0.418002 -0.210379       9        1   1.521126
25    -0.268713 -0.180075      10        1   1.398815
22     0.288663 -0.246456      11        1   1.240625
14    -0.434064 -0.033893      12        1   1.044404
9      0.449390 -0.016735      13        1   0.975467
11    -0.420932  0.127986      14        1   0.643045
18     0.393348  0.179248      15        1   0.383253
28    -0.087650  0.406223      16        1   0.315537
29    -0.290192  0.247384      17        1   0.287749
7     -0.194712  0.335091      18        1   0.232784
1      0.242743  0.026664      19        1   0.231704
6     -0.178590  0.092612      20        1   0.216775
27     0.257881  0.261249      21        1   0.213778
3      0.065533  0.395226      22        1   0.212876
12     0.175631  0.337904      23        1   0.188456
20    -0.139995  0.231289      24        1   0.172691
23     0.205343  0.145821      25        1   0.163539
13     0.034411  0.037828      26        1   0.153767
5     -0.043351  0.152622      27        1   0.140037
19     0.104164  0.220355      28        1   0.134854
2      0.011586  0.265791      29        1   0.131432
17     0.056938  0.135040      30        1   0.105614, topic_info=       Category          Freq         Term         Total  loglift  logprob
term                                                                      
59413   Default  11340.000000        новый  11340.000000  30.0000  30.0000
80499   Default   7649.000000        рубль   7649.000000  29.0000  29.0000
24184   Default  33568.000000          год  33568.000000  28.0000  28.0000
80194   Default  22122.000000       россия  22122.000000  27.0000  27.0000
22437   Default   4642.000000       газета   4642.000000  26.0000  26.0000
100970  Default  19865.000000      человек  19865.000000  25.0000  25.0000
23820   Default   6731.000000        глава   6731.000000  24.0000  24.0000
42111   Default   4859.000000     компания   4859.000000  23.0000  23.0000
105158  Default  33088.000000          это  33088.000000  22.0000  22.0000
25195   Default   5269.000000  государство   5269.000000  21.0000  21.0000
80161   Default  12922.000000   российский  12922.000000  20.0000  20.0000
94832   Default   8350.000000      украина   8350.000000  19.0000  19.0000
78370   Default   5541.000000      ребенок   5541.000000  18.0000  18.0000
88699   Default  10631.000000       страна  10631.000000  17.0000  17.0000
81358   Default   3067.000000         сайт   3067.000000  16.0000  16.0000
88192   Default   3589.000000       статья   3589.000000  15.0000  15.0000
103048  Default   2385.000000        школа   2385.000000  14.0000  14.0000
81669   Default   2883.000000      самолет   2883.000000  13.0000  13.0000
19762   Default   5523.000000      военный   5523.000000  12.0000  12.0000
72602   Default   8155.000000    президент   8155.000000  11.0000  11.0000
62566   Default   5593.000000  организация   5593.000000  10.0000  10.0000
26650   Default   4887.000000       данный   4887.000000   9.0000   9.0000
67440   Default   2309.000000       письмо   2309.000000   8.0000   8.0000
27386   Default  14505.000000         дело  14505.000000   7.0000   7.0000
89198   Default   7664.000000          суд   7664.000000   6.0000   6.0000
84417   Default   2493.000000        сирия   2493.000000   5.0000   5.0000
86503   Default   8924.000000     сообщать   8924.000000   4.0000   4.0000
51874   Default   2495.000000          млн   2495.0000

* #### TFIDF DTM

In [ ]:
tfidf_vectorizer = TfidfVectorizer(**vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(df.text_clear_lemma_1sentence)
print(dtm_tfidf.shape)

In [42]:
%%time

lda_tfidf = LatentDirichletAllocation(n_topics=30,               
                                      max_iter=2,                
                                      learning_method='online',
                                      random_state=42
                                     )

lda_tfidf.fit(dtm_tfidf)

/Users/trokhymenko/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 37.2 s, sys: 3.49 s, total: 40.7 s
Wall time: 41 s


In [46]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
22     0.389403  0.000233       1        1  89.967348
14    -0.012293 -0.064573       2        1   0.791658
4     -0.015126  0.005543       3        1   0.394776
7     -0.015263  0.005683       4        1   0.368280
15    -0.014238  0.003640       5        1   0.345679
25    -0.013785  0.003021       6        1   0.338195
26    -0.014227  0.003747       7        1   0.335821
3     -0.013611  0.002296       8        1   0.335778
8     -0.013706  0.002659       9        1   0.333860
21    -0.013578  0.002457      10        1   0.332515
29    -0.013665  0.002680      11        1   0.332027
18    -0.013577  0.002495      12        1   0.331531
27    -0.013492  0.002166      13        1   0.331159
0     -0.013232  0.001846      14        1   0.324212
13    -0.013124  0.001670      15        1   0.323155
24    -0.013179  0.001772      16        1   0.322418
11    -0.013162  0.001730      17        1   0.322011
28    -0.013094  0.001611      18        1   0.321898
23    -0.013085  0.001607      19        1   0.321468
10    -0.013099  0.001647      20        1   0.321257
9     -0.013126  0.001670      21        1   0.321038
17    -0.013087  0.001604      22        1   0.320731
1     -0.013094  0.001620      23        1   0.320684
19    -0.013083  0.001597      24        1   0.320357
12    -0.013080  0.001597      25        1   0.320357
20    -0.013076  0.001597      26        1   0.320357
6     -0.013081  0.001597      27        1   0.320357
5     -0.013078  0.001597      28        1   0.320357
16    -0.013082  0.001597      29        1   0.320357
2     -0.013081  0.001597      30        1   0.320357, topic_info=       Category        Freq              Term       Total  loglift  logprob
term                                                                       
59413   Default  155.000000             новый  155.000000  30.0000  30.0000
22437   Default   98.000000            газета   98.000000  29.0000  29.0000
26650   Default  118.000000            данный  118.000000  28.0000  28.0000
81358   Default   88.000000              сайт   88.000000  27.0000  27.0000
67440   Default   55.000000            письмо   55.000000  26.0000  26.0000
88192   Default   98.000000            статья   98.000000  25.0000  25.0000
85356   Default  101.000000            служба  101.000000  24.0000  24.0000
29428   Default  103.000000           должный  103.000000  23.0000  23.0000
105158  Default  300.000000               это  300.000000  22.0000  22.0000
35989   Default   77.000000           издание   77.000000  21.0000  21.0000
49508   Default   52.000000          материал   52.000000  20.0000  20.0000
72059   Default   81.000000             право   81.000000  19.0000  19.0000
105595  Default   81.000000          являться   81.000000  18.0000  18.0000
96749   Default   70.000000       федеральный   70.000000  17.0000  17.0000
37912   Default   65.000000              июнь   65.000000  16.0000  16.0000
85381   Default   73.000000            случай   73.000000  15.0000  15.0000
101669  Default   72.000000             число   72.000000  14.0000  14.0000
62484   Default   61.000000    опубликовывать   61.000000  13.0000  13.0000
78562   Default   40.000000          редакция   40.000000  12.0000  12.0000
59394   Default   59.000000           новость   59.000000  11.0000  11.0000
8006    Default   41.000000             автор   41.000000  10.0000  10.0000
54866   Default   54.000000        направлять   54.000000   9.0000   9.0000
55260   Default   51.000000         настоящий   51.000000   8.0000   8.0000
47833   Default   48.000000             любой   48.000000   7.0000   7.0000
26646   Default   42.000000            данные   42.000000   6.0000   6.0000
49428   Default   40.000000          массовый   40.000000   5.0000   5.0000
33193   Default   38.000000  законодательство   38.000000   4.0000   4.0000
94779   Default   31.0000

In [50]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
22     -27.953764  -23.630526       1        1  89.967348
14     -10.183451   20.380186       2        1   0.791658
4       50.987865  -92.469727       3        1   0.394776
7        8.507343   93.447128       4        1   0.368280
15     -84.621078  -32.808899       5        1   0.345679
25     -86.553482   61.779457       6        1   0.338195
26      27.943628   46.929684       7        1   0.335821
3     -111.171738  117.244278       8        1   0.335778
8      -94.375511 -103.181808       9        1   0.333860
21       6.942490  147.680328      10        1   0.332515
29     -48.545578  -67.402763      11        1   0.332027
18      23.418604  -12.354615      12        1   0.331531
27      65.916061  128.910446      13        1   0.331159
0      121.733955   50.631466      14        1   0.324212
13       5.827951  -63.888992      15        1   0.323155
24     -60.439144   14.935225      16        1   0.322418
11     -50.231560  105.092468      17        1   0.322011
28      69.242294   16.756954      18        1   0.321898
23     -34.178757 -116.549500      19        1   0.321468
10      70.783760   75.234726      20        1   0.321257
9       17.917316 -136.484161      21        1   0.321038
17    -164.768768   -0.956554      22        1   0.320731
1     -134.808228  -56.618446      23        1   0.320684
19      63.437244  -38.668747      24        1   0.320357
12     108.266602  -76.095810      25        1   0.320357
20     -32.658127   60.082783      26        1   0.320357
6      118.817917  -12.674689      27        1   0.320357
5     -145.184494   63.348347      28        1   0.320357
16    -111.322105   12.422539      29        1   0.320357
2      -54.816601  158.818039      30        1   0.320357, topic_info=       Category        Freq              Term       Total  loglift  logprob
term                                                                       
59413   Default  155.000000             новый  155.000000  30.0000  30.0000
22437   Default   98.000000            газета   98.000000  29.0000  29.0000
26650   Default  118.000000            данный  118.000000  28.0000  28.0000
81358   Default   88.000000              сайт   88.000000  27.0000  27.0000
67440   Default   55.000000            письмо   55.000000  26.0000  26.0000
88192   Default   98.000000            статья   98.000000  25.0000  25.0000
85356   Default  101.000000            служба  101.000000  24.0000  24.0000
29428   Default  103.000000           должный  103.000000  23.0000  23.0000
105158  Default  300.000000               это  300.000000  22.0000  22.0000
35989   Default   77.000000           издание   77.000000  21.0000  21.0000
49508   Default   52.000000          материал   52.000000  20.0000  20.0000
72059   Default   81.000000             право   81.000000  19.0000  19.0000
105595  Default   81.000000          являться   81.000000  18.0000  18.0000
96749   Default   70.000000       федеральный   70.000000  17.0000  17.0000
37912   Default   65.000000              июнь   65.000000  16.0000  16.0000
85381   Default   73.000000            случай   73.000000  15.0000  15.0000
101669  Default   72.000000             число   72.000000  14.0000  14.0000
62484   Default   61.000000    опубликовывать   61.000000  13.0000  13.0000
78562   Default   40.000000          редакция   40.000000  12.0000  12.0000
59394   Default   59.000000           новость   59.000000  11.0000  11.0000
8006    Default   41.000000             автор   41.000000  10.0000  10.0000
54866   Default   54.000000        направлять   54.000000   9.0000   9.0000
55260   Default   51.000000         настоящий   51.000000   8.0000   8.0000
47833   Default   48.000000             любой   48.000000   7.0000   7.0000
26646   Default   42.000000            данные   42.000000   6.0000   6.0000
49428   Default   40.000000          массовый   4

In [51]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
22     0.109058  0.373830       1        1  89.967348
14     0.056028 -0.037114       2        1   0.791658
4     -0.010909 -0.012740       3        1   0.394776
7     -0.007788 -0.007323       4        1   0.368280
15    -0.003316 -0.013263       5        1   0.345679
25    -0.004138 -0.012459       6        1   0.338195
26    -0.005070 -0.009606       7        1   0.335821
3     -0.006335 -0.013035       8        1   0.335778
8     -0.007002 -0.011758       9        1   0.333860
21    -0.005116 -0.011606      10        1   0.332515
29    -0.004537 -0.011640      11        1   0.332027
18    -0.006320 -0.011428      12        1   0.331531
27    -0.006407 -0.012333      13        1   0.331159
0     -0.005900 -0.012124      14        1   0.324212
13    -0.005754 -0.012429      15        1   0.323155
24    -0.005874 -0.012238      16        1   0.322418
11    -0.005858 -0.012390      17        1   0.322011
28    -0.005776 -0.012316      18        1   0.321898
23    -0.005743 -0.012325      19        1   0.321468
10    -0.005701 -0.012264      20        1   0.321257
9     -0.005800 -0.012389      21        1   0.321038
17    -0.005746 -0.012355      22        1   0.320731
1     -0.005728 -0.012283      23        1   0.320684
19    -0.005757 -0.012343      24        1   0.320357
12    -0.005757 -0.012340      25        1   0.320357
20    -0.005751 -0.012348      26        1   0.320357
6     -0.005746 -0.012342      27        1   0.320357
5     -0.005757 -0.012342      28        1   0.320357
16    -0.005749 -0.012349      29        1   0.320357
2     -0.005754 -0.012347      30        1   0.320357, topic_info=       Category        Freq              Term       Total  loglift  logprob
term                                                                       
59413   Default  155.000000             новый  155.000000  30.0000  30.0000
22437   Default   98.000000            газета   98.000000  29.0000  29.0000
26650   Default  118.000000            данный  118.000000  28.0000  28.0000
81358   Default   88.000000              сайт   88.000000  27.0000  27.0000
67440   Default   55.000000            письмо   55.000000  26.0000  26.0000
88192   Default   98.000000            статья   98.000000  25.0000  25.0000
85356   Default  101.000000            служба  101.000000  24.0000  24.0000
29428   Default  103.000000           должный  103.000000  23.0000  23.0000
105158  Default  300.000000               это  300.000000  22.0000  22.0000
35989   Default   77.000000           издание   77.000000  21.0000  21.0000
49508   Default   52.000000          материал   52.000000  20.0000  20.0000
72059   Default   81.000000             право   81.000000  19.0000  19.0000
105595  Default   81.000000          являться   81.000000  18.0000  18.0000
96749   Default   70.000000       федеральный   70.000000  17.0000  17.0000
37912   Default   65.000000              июнь   65.000000  16.0000  16.0000
85381   Default   73.000000            случай   73.000000  15.0000  15.0000
101669  Default   72.000000             число   72.000000  14.0000  14.0000
62484   Default   61.000000    опубликовывать   61.000000  13.0000  13.0000
78562   Default   40.000000          редакция   40.000000  12.0000  12.0000
59394   Default   59.000000           новость   59.000000  11.0000  11.0000
8006    Default   41.000000             автор   41.000000  10.0000  10.0000
54866   Default   54.000000        направлять   54.000000   9.0000   9.0000
55260   Default   51.000000         настоящий   51.000000   8.0000   8.0000
47833   Default   48.000000             любой   48.000000   7.0000   7.0000
26646   Default   42.000000            данные   42.000000   6.0000   6.0000
49428   Default   40.000000          массовый   40.000000   5.0000   5.0000
33193   Default   38.000000  законодательство   38.000000   4.0000   4.0000
94779   Default   31.0000